## Unemployment and COVID

Brian Dew, July 25, 2019

#### CPS Documentation:
- [BLS Handbook of Methods](https://www.bls.gov/opub/hom/cps/home.htm)
- [2020 Data Dictionary](https://www2.census.gov/programs-surveys/cps/datasets/2020/basic/2020_Basic_CPS_Public_Use_Record_Layout_plus_IO_Code_list.txt)
- [2017 Data Dictionary](https://www2.census.gov/programs-surveys/cps/datasets/2017/basic/January_2017_Record_Layout.txt) (covers 2019 data)


In [1]:
import pandas as pd
import numpy as np
import os
import re
import struct
import gzip
import shutil
from io import BytesIO
import requests

### Download Files

In [2]:
# Download monthly CPS files if not available in current directory
file_list = ['apr19pub.dat', 'may19pub.dat', 'jun19pub.dat', 
             'apr20pub.dat', 'may20pub.dat', 'jun20pub.dat']
for file in file_list:
    if file not in os.listdir():
        file_loc = f'https://www2.census.gov/programs-surveys/cps/datasets/20{file[3:5]}/basic/{file}.gz'
        print(f'Downloading: {file} from census.gov')
        r = requests.get(file_loc)
        with gzip.open(BytesIO(r.content), 'r') as f_in, open(file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            
# Download two data dictionaries if not available in current directory
dd_list = {'2020_Basic_CPS_Public_Use_Record_Layout_plus_IO_Code_list.txt': 2020, 
             'January_2017_Record_Layout.txt': 2017}
for file, year in dd_list.items():
    if file not in os.listdir():
        file_loc = f'https://www2.census.gov/programs-surveys/cps/datasets/{year}/basic/{file}'
        print(f'Downloading: {file} from census.gov')
        r = requests.get(file_loc)
        with open(file, 'wb') as f:
            f.write(r.content)

### Read data

Search the data dictionary for `var_names` and pull those columns from fixed-width format microdata file. 

In [3]:
# manually list out the IDs for series of interest 
var_names = ['HRMONTH', 'HRYEAR4', 'PRTAGE', 'PRFAMNUM', 'PRDISFLG', 'QSTNUM', 
             'PEMLR', 'PEHRACTT', 'PEIO1COW', 'PEDWRSN', 'PRWNTJOB', 'PENLFACT',
             'PEDWWNTO', 'PRWKSTAT', 'PESCHENR', 'PWSSWGT', 'PWFMWGT', 'HWHHWGT']

dd_list = {19: 'January_2017_Record_Layout.txt',
           20: '2020_Basic_CPS_Public_Use_Record_Layout_plus_IO_Code_list.txt'}

unpackers = {}
for year, dd in dd_list.items():
    # read data dictionary text file 
    data_dict = open(dd, 'r', encoding='iso-8859-1').read()
    # regular expression matching series name and data dict pattern
    p = f'\n({"|".join(var_names)})\s+(\d+)\s+.*?\t+.*?(\d\d*).*?(\d\d+)'
    # dictionary of variable name: [start, end, and length + 's']
    d = {s[0]: [int(s[2])-1, int(s[3]), f'{s[1]}s']
         for s in re.findall(p, data_dict)}
    # lists of variable starts, ends, and lengths
    start, end, width = zip(*d.values())
    # create list of which characters to skip in each row
    skip = ([f'{s - e}x' for s, e in zip(start, [0] + list(end[:-1]))])
    # create format string by joining skip and variable segments
    unpack_fmt = ''.join([j for i in zip(skip, width) for j in i])
    # struct to read each row of the file using the format string
    unpackers[year] = struct.Struct(unpack_fmt).unpack_from    

Open the six monthly files as a pandas dataframe, keep only rows with a person weight greater than zero

In [4]:
df = pd.DataFrame()

for file in file_list:
    raw_data = open(file, 'rb').readlines()

    wgt = d['PWSSWGT']  # Person sample weight
    # unpack and store data of interest if sample weight > 0
    data = [[*map(int, unpackers[int(file[3:5])](row))] for row in raw_data
            if int(row[wgt[0]:wgt[1]]) > 0]
    
    df = df.append(pd.DataFrame(data, columns=d.keys()))

### Create Variables

Categorize person records into various labor market groups and categories that relate to possible work limitations. Summarize these groups and categories at the family level.

In [5]:
# Uniquely identify households and families using these variables
hh_grp = ['HRYEAR4', 'HRMONTH', 'QSTNUM']
fam_grp = ['HRYEAR4', 'HRMONTH', 'QSTNUM', 'PRFAMNUM']

groups = ['U18', 'DIS', 'STU', 'CARE', 'UNEM', 'UNDE', 'ELD', 'SE']

# Reason for not being in labor force
nilfreason = lambda x: pd.Categorical(
    np.where((x['PRWNTJOB']==2) & 
             ((x['PEMLR']==6) | (x['PENLFACT'].isin([1, 2]))), 
             'Disabled/Ill',
    np.where((x['PRWNTJOB']==2) & (x['PENLFACT']==4), 'Family',
    np.where((x['PRWNTJOB']==2) & ((x['PEMLR']==5) | (x['PENLFACT']==5)), 
             'Retired',
    np.where((x['PRWNTJOB']==2) & (x['PENLFACT']==3), 'School',
    np.where(x['PEDWWNTO']==1, 'Discouraged',
    np.where(x['PEMLR'].isin([5, 6, 7]), 'Other', np.nan)))))))
# Part time for economic reasons
ptecon = lambda x: pd.Categorical(
    np.where(x['PRWKSTAT'].isin([3, 6]), 1, 
    np.where(x['PRWKSTAT'].between(2, 10), 0, np.nan)))
# Marginally attached to labor force
mrgnatt = lambda x: pd.Categorical(
    np.where(x['PEDWRSN'].between(1, 11), 1, 0))
lfs = lambda x: pd.Categorical(
    np.where(x['PEMLR'].isin([1, 2]), 'Employed',
    np.where(x['PEMLR'].isin([3, 4]), 'Unemployed',
    np.where(x['PEMLR'].isin([5, 6, 7]), 'NILF', np.nan))))

df = df.assign(NILFREASON = nilfreason, PTECON = ptecon, 
               MRGNATT = mrgnatt, LFS = lfs)

# Weights have four implied decimal places:
for weight in ['PWFMWGT', 'PWSSWGT', 'HWHHWGT']:
    df[weight] = df[weight] / 10000.0

# File identifies non-workers as having -1 hours, which affects average
df['PEHRACTT'].replace(-1, 0, inplace=True)

# Identify categories of people less likely to recieve labor income
# Children
df['U18'] = np.where(df['PRTAGE'] < 18, 1, 0)
# Disability or Illness
df['DIS'] = np.where((df['PRDISFLG'] == 1) | (df['NILFREASON'] == 'Disabled/Ill'), 1, 0)
# Students
df['STU'] = np.where((df['PESCHENR'] == 1) | (df['NILFREASON'] == 'School'), 1, 0)
# Unpaid Caregivers
df['CARE'] = np.where(df['NILFREASON'] == 'Family', 1, 0)
# Unemployed
df['UNEM'] = np.where(df['LFS'] == 'Unemployed', 1, 0)
# Underutilized
df['UNDE'] = np.where((df['LFS'] == 'Unemployed') | (df['MRGNATT'] == 1) | 
                      (df['PTECON'] == 1) | 
                      (df['NILFREASON'] == 'Discouraged'), 1, 0)
# Elderly
df['ELD'] = np.where(df['PRTAGE'] > 64, 1, 0)
# Self-employed
df['SE'] = np.where(df['PEIO1COW'].isin([6,7]), 1, 0)

# Family level summary
df['FMHRST'] = df.groupby(fam_grp)['PEHRACTT'].transform('sum')
df['FMNUM'] = df.groupby(fam_grp)['PWSSWGT'].transform('count')
df['FMHRSPP'] = df['FMHRST'] / df['FMNUM']
for group in groups:
    df['FM' + group] = df.groupby(fam_grp)[group].transform('sum')
df['FMANY'] = df[['FM' + group for group in groups]].sum(axis=1)
df['FMTOT'] = 1
df['FMNONE'] = np.where(df['FMANY'] == 0, 1, 0)

#### Create Table 1

In [6]:
# Take first row from each family
fam_data = df.groupby(fam_grp).nth(0).reset_index()

names = {'FMTOT': 'Total, all families', 
         'Label': 'Family contains one or more: ',
         'FMU18': '  Children', 
         'FMDIS': '  Persons with disabilities', 
         'FMSTU': '  Students',
         'FMCARE': '  Unpaid caregivers', 
         'FMUNEM': '  Unemployed', 
         'FMUNDE': '  Underutilized',
         'FMELD': '  Elderly', 
         'FMSE': '  Self-employed',
         'FMANY': '  Family contains any from above groups',
         'FMNONE': '  Family contains none from above groups'}

idx = [value for key, value in names.items()]

col = pd.MultiIndex.from_product([['April to June 2020', 'April to June 2019'],
                                  ['Share of families', 'Average weekly hours worked per family member']])

tbl = pd.DataFrame('', idx, col)

for year in [2020, 2019]:
    data = fam_data.query('HRYEAR4 == @year')
    num = data.PWFMWGT.sum() / 3
    date_col = f'April to June {year}'
    for group in names.keys():
        if group != 'Label':
            d = data.loc[data[group] > 0]
            n = d.PWFMWGT.sum () / 3
            hrs = f"{np.average(d['FMHRSPP'], weights=d['PWFMWGT']):.1f}"
            share = f'{(n/num)*100:.1f}%'
        else:
            hrs = ''
            share = ''
        col = (date_col, 'Average weekly hours worked per family member')
        tbl.loc[names[group], col] = hrs
        col = (date_col, 'Share of families')
        tbl.loc[names[group], col] = share
print('Table 1. Actual weekly hours per family member')       
tbl

Table 1. Actual weekly hours per family member


April to June 2020  \
                                          Share of families   
Total, all families                                  100.0%   
Family contains one or more:                                  
  Children                                            27.8%   
  Persons with disabilities                           21.4%   
  Students                                            15.2%   
  Unpaid caregivers                                    9.0%   
  Unemployed                                          12.4%   
  Underutilized                                       22.5%   
  Elderly                                             29.5%   
  Self-employed                                       10.4%   
  Family contains any from above groups               80.3%   
  Family contains none from above groups              19.7%   

                                                                                        \
                                         Average weekly hours worked per family member   
Total, all families                                                               16.0   
Family contains one or more:                                                             
  Children                                                                        12.6   
  Persons with disabilities                                                        7.1   
  Students                                                                        14.4   
  Unpaid caregivers                                                                9.1   
  Unemployed                                                                       7.8   
  Underutilized                                                                   10.3   
  Elderly                                                                          6.5   
  Self-employed                                                                   20.8   
  Family contains any from above groups                                           11.5   
  Family contains none from above groups                                          34.6   

                                         April to June 2019  \
                                          Share of families   
Total, all families                                  100.0%   
Family contains one or more:                                  
  Children                                            28.2%   
  Persons with disabilities                           22.3%   
  Students                                            15.7%   
  Unpaid caregivers                                    9.0%   
  Unemployed                                           3.8%   
  Underutilized                                        9.5%   
  Elderly                                             28.3%   
  Self-employed                                       10.1%   
  Family contains any from above groups               76.2%   
  Family contains none from above groups              23.8%   

                                                                                        
                                         Average weekly hours worked per family member  
Total, all families                                                               19.5  
Family contains one or more:                                                            
  Children                                                                        14.9  
  Persons with disabilities                                                        8.7  
  Students                                                                        17.6  
  Unpaid caregivers                                                               10.5  
  Unemployed                                                                       9.1  
  Underutilized                                                                   12.6  
  Elderly                                                                          8.2  
  Self-employed                    

#### Create table 2

In [7]:
names = {'FMTOT': 'Unemployed group and family', 
         'Label': 'Family members include: ',
         'U18': '  Children', 
         'DIS': '  Persons with disabilities', 
         'STU': '  Students',
         'CARE': '  Unpaid caregivers', 
         'ELD': '  Elderly'}

idx = ['Unemployed group alone'] + [value for key, value in names.items()]

col = pd.MultiIndex.from_product([['April to June 2020', 'April to June 2019'],
                                  ['Unemployed', 'Underutilized']])

tbl = pd.DataFrame('', idx, col)

for year in [2020, 2019]:
    date_col = f'April to June {year}'
    for category in ['FMUNEM', 'FMUNDE']:
        col_name = 'Unemployed' if category == 'FMUNEM' else 'Underutilized'
        data = df.loc[(df['HRYEAR4'] == year) & (df[category] > 0)]
        group_size = f'{(data.loc[data[category[2:]] == 1].PWSSWGT.sum()/3).round(-3):,.0f}'
        col = (date_col, col_name)
        tbl.loc['Unemployed group alone', col] = group_size
        for key, value in names.items():
            if key != 'Label':
                tot = f'{(data.loc[data[key] == 1].PWSSWGT.sum()/3).round(-3):,.0f}'
            else: 
                tot = ''
            tbl.loc[value, col] = tot

print('Table 2. Composition of unemployed and underutilized families')            
tbl

Table 2. Composition of unemployed and underutilized families


April to June 2020                \
                                    Unemployed Underutilized   
Unemployed group alone              20,536,000    39,340,000   
Unemployed group and family         49,206,000    87,764,000   
Family members include:                                        
  Children                          11,335,000    20,748,000   
  Persons with disabilities          3,231,000     6,066,000   
  Students                           5,863,000    10,275,000   
  Unpaid caregivers                  1,147,000     2,291,000   
  Elderly                            3,264,000     6,234,000   

                            April to June 2019                
                                    Unemployed Underutilized  
Unemployed group alone               5,861,000    15,274,000  
Unemployed group and family         15,563,000    37,915,000  
Family members include:                                       
  Children                           3,806,000     9,096,000  
  Persons with disabilities          1,347,000     3,480,000  
  Students                           1,943,000     4,877,000  
  Unpaid caregivers                    267,000       798,000  
  Elderly                              951,000     2,951,000